In [ ]:
import pandas as pd
import numpy as np

class RSI2Trader:
    def __init__(self, portfolio, fees, tax, capital):
        self.portfolio = portfolio  # DataFrame with allocations
        self.fees = fees  # dict with 'buy' and 'sell' percentages
        self.tax = tax  # transaction tax percentage
        self.capital = capital
        self.positions = {item: 0 for item in portfolio['Item']}  # track positions

    def compute_rsi2(self, prices):
        delta = prices.diff()
        gain = delta.clip(lower=0)
        loss = -delta.clip(upper=0)
        avg_gain = gain.rolling(window=2).mean()
        avg_loss = loss.rolling(window=2).mean()
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        return rsi

    def generate_signals(self, price_series):
        rsi = self.compute_rsi2(price_series)
        buy_signal = (rsi < 20) & (rsi.shift(1) >= 20)
        sell_signal = (rsi > 40) & (rsi.shift(1) <= 40)
        return buy_signal, sell_signal

    def execute_trade(self, item, price, buy_signal, sell_signal):
        max_allocation = self.portfolio.loc[self.portfolio['Item'] == item, 'final_alloc_value'].values[0]
        current_position = self.positions[item]

        if buy_signal and current_position == 0:
            qty = max_allocation / price
            cost = qty * price * (1 + self.fees['buy'])
            if cost <= self.capital:
                self.capital -= cost
                self.positions[item] = qty
                print(f"Bought {qty:.2f} of {item} at {price}")
        elif sell_signal and current_position > 0:
            revenue = current_position * price * (1 - self.fees['sell'] - self.tax)
            self.capital += revenue
            self.positions[item] = 0
            print(f"Sold {current_position:.2f} of {item} at {price}")

    def run_day(self, market_data_day):
        for item in self.positions.keys():
            prices = market_data_day[item]
            buy_signal, sell_signal = self.generate_signals(prices)
            # For simplicity, use last price and signals
            if buy_signal.iloc[-1]:
                self.execute_trade(item, prices.iloc[-1], True, False)
            elif sell_signal.iloc[-1]:
                self.execute_trade(item, prices.iloc[-1], False, True)

# Usage would include loading your portfolio, fetching daily price data per item,
# then running this loop per day, updating capital and positions.

